In [3]:
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
from PIL import Image
import os
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [5]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((255,255)),
    torchvision.transforms.CenterCrop((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
model = models.vgg16(pretrained = True)

In [7]:
with open("/content/sample_data/imagenet_class_index.txt","r") as f:
  lines = f.readlines()

class_idx_dict = {}
for line in lines:
  line = line.strip()
  key , value = line.split(':')
  key = key.strip()
  value = value.strip()
  class_idx_dict[key] = value


class_idx = [class_idx_dict[f"{i}"] for i in class_idx_dict.keys()]

In [8]:
for params in model.parameters():
  params.requires_grad == False

In [ ]:
model.to(device)

In [ ]:
model.eval()

In [11]:
# for name, layer in model.named_modules():
#   if isinstance(layer, torch.nn.Conv2d):
#     weights = layer.weight.shape
#     bias = layer.bias.shape
#     name = name.replace('features.','')
#     print(f"Conv Layer {name}")
#     print(f"Weights Shape : {weights}")
#     print(f"Bias Shape : ", bias if bias is not None else None )
#     print('-'*40)

In [12]:
name_list = {}
i=1
for name, layer in model.features.named_children():
  if isinstance(layer, torch.nn.Conv2d):
    name_list[i] = int(name)
    i += 1


In [ ]:
print(name_list)

In [ ]:
user_input = int(input("Enter the Conv Layer which you need to see : "))
conv_layer_num = name_list[user_input]
conv_layer_name = "Conv_Layer" + str(conv_layer_num)

In [35]:
# Takes the weights and bias of the Particular Layer as specified by the User
for name, layer in model.features.named_children():
  if int(name) == conv_layer_num:
    filter, bias = layer.weight, layer.bias

# Normalizes the filter
f_max, f_min = filter.max(), filter.min()
filter = (filter - f_min) / (f_max - f_min)

In [ ]:
total_images = int(input("Enter the no of images that has to be displayed : "))


In [ ]:
num_filter = filter.data.size(0)
filter_image_list = [] # List of all the filter images

# Let's say in_channel is 3 and out_channel is 10.
# Then each in_channel has total no of 10 filters, so totally 30 filters.
# Looping through every filter and adding each filter image to list.
# each in_channel consist of no of out_channels of filters

for i in range(num_filter):
  filter_data = filter.data[i].cpu().numpy().transpose(1,2,0)
  num_channels = filter_data.shape[2]
  for j in range(num_channels):
    filter_image_list.append(filter_data[:,:,j])

len_filter_image_list = total_images #len(filter_image_list) #Total no of filters

images_per_row = 6  #Specifying the required no of images per row
num_rows = (len_filter_image_list + images_per_row - 1) // images_per_row
num_cols = images_per_row

# Create a figure and axes
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 2*num_rows))

# Flatten the axes array if needed
if len_filter_image_list == 1:
    axes = np.array([[axes]])

# Loop through the images and axes to plot them
for i, ax in enumerate(axes.flat):
    if i < len_filter_image_list:
        ax.imshow(filter_image_list[i], cmap='gray')  # Adjust the colormap as needed
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.axis('off')  # Turn off extra axes

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [45]:
# Feature Maps

conv_layers = {}

def get_conv_layers(name):
    def hook(module, input, output):
        conv_layers[name] = output.detach()
    return hook

# Register a forward hook to capture the output of the desired layer (conv1)
model.features[conv_layer_num].register_forward_hook(get_conv_layers(conv_layer_name))

# Gets the Image and Process it
image = Image.open('/content/sample_data/french_bulldog.jpeg')
transformed_image = transform(image)
final_image = transformed_image.unsqueeze(0)
final_image = final_image.to(device)

# Model Prediction
output = model(final_image)

# Extracts the Required Conv Layer from the list of Conv Layers (conv_layers ==> {'Conv1, 'Conv2', .....})
final_output = conv_layers[conv_layer_name].squeeze(0) # removes the batch value

# Converts the Tensor to Numpy format
final_data = final_output.cpu().numpy()
final_data = final_data.transpose(1,2,0) # [(64,224, 224) == > (224, 224, 64)]
s = final_data.shape

In [ ]:
print(s)

In [ ]:
# Displaying the Feature Maps
total_no_images = total_images #s[2]

images_per_row = 6  #Specifying the required no of images per row
num_rows = (total_no_images + images_per_row - 1) // images_per_row
num_cols = images_per_row

# Create a figure and axes
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))

# Flatten the axes array if needed
if total_no_images == 1:
    axes = np.array([[axes]])

# Loop through the images and axes to plot them
for i, ax in enumerate(axes.flat):
    if i < total_no_images:
        ax.imshow(final_data[:,:,i], cmap='gray')  # Adjust the colormap as needed
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.axis('off')  # Turn off extra axes

# Adjust layout and display the plot
plt.tight_layout()
plt.show()